In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import math
import copy
import random
import time

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
univseed = 42

random.seed(univseed)
np.random.seed(univseed)
torch.manual_seed(univseed)
torch.cuda.manual_seed(univseed)
torch.backends.cudnn.deterministic = True

In [5]:
# mu = [0.48093379,0.44808328,0.39650237]; sd = [0.22996924,0.22610814,0.22566715];
# transf = transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mu,sd)])
# totdata = datasets.ImageFolder()

# datapath = "/content/drive/MyDrive/A2_data/"
# traindata = np.load(datapath + "train_x.npy", mmap_mode='r')
# trainlabels = np.load(datapath + "train_y.npy", mmap_mode='r')
# testdata = np.load(datapath + "test_x.npy", mmap_mode='r')
# testlabels = np.load(datapath + "test_y.npy", mmap_mode='r')
# # img1 = []
# # img2 = []
# # img3 = []
# # for lab in traindata:
# #   img1.append(lab[0])
# #   img2.append(lab[1])
# #   img3.append(lab[2])

# # means = [np.mean(img1),np.mean(img2),np.mean(img3)]
# # stdevs = [np.std(img1),np.std(img2),np.std(img3)]
# # print(means)
# # print(stdevs)

# mean = np.mean(traindata) / 255
# std = np.std(traindata) / 255
# print(mean)
# print(std)
# transformations = transforms.Compose([
#     transforms.Resize(255),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[122.63811599026143, 114.26123747517829, 101.10810390651845], std=[70.52046150278801, 68.46892052533573, 71.82977751494695])])

# trainlabels1 = []

# for label in trainlabels:
#   v = np.zeros(shape=(200,))
#   #print(label)
#   v[label] = 1
#   trainlabels1.append(v)
#   #print(v)

# #  break
# # trainlabels = np.asarray(trainlabels1)
# # del trainlabels1
# # print(trainlabels.shape)
# testlabels1 = []
# for label in testlabels:
#   v = np.zeros(shape=(200,))
#   v[label] = 1
#   testlabels1.append(v)

# # testabels = np.asarray(testlabels1)
# # del testlabels1
# # print(testabels.shape)

In [ ]:
# transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor(),])
transform = transforms.Compose([transforms.ToTensor()])
mnist = datasets.MNIST('/tmp', download=True, transform=transform)
mnist_loader = torch.utils.data.DataLoader(dataset=mnist,
                                            batch_size=21,
                                            shuffle=True,
                                            num_workers=2)

mnist_test = datasets.MNIST('/tmp',train=False,transform=transform)
mnist_test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                            batch_size=21,
                                            shuffle=True,
                                            num_workers=2)

In [ ]:
transform_sv = transforms.Compose([transforms.ToTensor(),transforms.Grayscale(1),transforms.Resize((28,28))]) #change normalize values if need be

svhn = datasets.SVHN('/tmp', transform=transform_sv,download=True,split='train')
svhn_test = datasets.SVHN('/tmp', split='test', transform=transform_sv,download=True)

#mnist = torchvision.datasets.MNIST('/tmp', download=True, transform=transform)

svhn_loader = torch.utils.data.DataLoader(dataset=svhn,
                                          batch_size=21,
                                          shuffle=True,
                                          num_workers=2)

svhn_testload = torch.utils.data.DataLoader(dataset=svhn_test,
                                            batch_size=21,
                                            shuffle=True,
                                            num_workers=2)

In [ ]:
# print(mnist_test_loader)

In [12]:
# dataiter = iter(mnist_loader)

In [16]:
# validation_set = 0.1

# n_train = int(len(traindata) * (1-validation_set))
# n_valid = len(traindata) - n_train

# validdata = traindata[:n_valid]
# validlabels = trainlabels1[:n_valid]

# traindata1 = traindata[n_valid:]
# trainlabels1 = trainlabels1[n_valid:]

In [ ]:
class lenet(nn.Module):
    def __init__(self):
        super(lenet, self).__init__()
        self.c1 = nn.Conv2d(1, 6, 5, padding=2)
        self.c2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
                
    def forward(self, x):
        x = F.relu(self.c1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.c2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

    
model = lenet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using device:', device)
model.to(device)

In [7]:
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.99))
#lrt = 0.001; momt = 0;
#optimizer = optim.SGD(model.parameters(), lr=lrt,momentum=momt)

In [2]:
epoch=10

In [9]:
def findacc(testloader,net):
  correct = 0
  total = 0
  with torch.no_grad():
    for data in testloader:
 #     print(data.type())
      images, labels = data
      outputs = net(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  return float(correct/total)

In [ ]:
trainaccs = []
testaccs = []
losses = []
nonzerowts = []
identifier = None

if(identifier=="svhnTrain"):
  data_ld = svhn_loader
else:
  data_ld = mnist_loader

for e in range(epoch):
  tot_loss = 0.0
  iv=0

  for i, data in enumerate(data_ld, 0):
    inputs, labels = data
   # inputs, labels = inputs.cuda(), labels.cuda() # add this line

    optimizer.zero_grad()
    #print(inputs.shape)
    inputs1 = inputs

    if(identifier=="3.3Algo2"):
      btch = inputs.shape[0]
      subset = torch.utils.data.Subset(svhn, np.random.choice(len(svhn),btch))
      testloader_subset = torch.utils.data.DataLoader(subset, batch_size=btch, num_workers=2, shuffle=False)
      
      for i,data in enumerate(testloader_subset,0):
        vhnv = data[0]

      inputs1 = torch.maximum(inputs,(np.random.randint(1,10,1)[0]/11)*vhnv) #change to 0.5*vhnv for other exp

    outputs = model(inputs1)
    loss = criterion(outputs, labels)
    loss2 = loss

    if(identifier=="3.3Algo1"):
      btch = outputs.data.shape[0]
      subset = torch.utils.data.Subset(svhn, np.random.choice(len(svhn),btch))
      testloader_subset = torch.utils.data.DataLoader(subset, batch_size=btch, num_workers=2, shuffle=False)
      for i,data in enumerate(testloader_subset,0):
        loss1 = criterion(model(data[0]),labels)
 
      loss2 = 0.95*loss + 0.05*loss1

    loss2.backward()
    optimizer.step()
    tot_loss += loss2.item()
    iv+=1

  print('[%d, %5d] loss: %.3f' %(e + 1, i + 1, tot_loss /iv))
  losses.append(tot_loss/iv)
  
  trainaccs.append(findacc(data_ld,model))

  if identifier is None:
    testaccs.append(findacc(mnist_loader,model))
  elif(identifier=="svhnTrain"):
    testaccs.append(findacc(mnist_loader,model))
    
  else:
    testaccs.append(findacc(svhn_loader,model))
  
  zeros = 0
  allz = 0
  for param in model.parameters():
    if param is not None:
      zeros += param.nonzero().size(0)
      allz+=param.numel()
  nonzerowts.append(float(zeros/allz))

In [ ]:
# print(trainaccs)
# print(losses)
# print(testaccs)
# print(nonzerowts)

In [ ]:
# ------plots------------
# import matplotlib.pyplot as plt

# plt.plot(range(10),tr1[:10],label="SVHN 0.5")
# plt.plot(range(len(tr2)),tr2,label="SVHN no use")
# plt.plot(range(len(tr3)),tr3,label="SVHN random wt")

# plt.title("LeNet, cross entropy, Adam")
# plt.ylabel("Accuracies")
# plt.xlabel("Epochs")
# plt.legend()

In [ ]:

# plt.plot(range(epoch),trainaccs,label="MNIST train accuracy")
# plt.plot(range(epoch),testaccs,label="SVHN train accuracy")
# plt.title("LeNet, cross entropy, Adam")
# plt.ylabel("Accuracies")
# plt.xlabel("Epochs")
# plt.legend()

In [ ]:
# plt.plot(range(epoch),losses,label="train accuracy")
# # plt.plot(range(epoch),testaccs,label="test accuracy")
# plt.title("LeNet, cross entropy, Adam")
# plt.ylabel("Loss")
# plt.xlabel("Epochs")
# # plt.legend()